# Práctica 3 - Ensembles y selección de variables Minería de Datos 2016/2017 - Rubén Donate Serrano

En esta práctica tendremos dos objetivos principales:

### A) Uso y ajuste de métodos basados en ensembles
Al igual que en la práctica anterior se busca que el alumno aprenda a utilizar los 4 modelos de ensemble vistos y a compararlos con el algoritmo base (especialmente árboles de decisión), para ello se plantean los siguientes ejercicios:

* **A.1) Implementación básica de un ensemble de manera manual:**
    ```
        Pseudocodigo:
        Generar una lista de datos muestreados (con/sin remplazo, con/sin muestreo de atributos)
        Aprender un clasificador para cada muestra
        Obtener predicciones para cada modelo
        Obtener la moda de dichas predicciones
    ```
    
*  **A.2) Uso de Bagging, Boosting, Random Forest y Gradient Boosting**: Aprender y ajustar los parámetros de los distintos modelos para maximizar la clasificación. Comparar estos resultados con el ensemble implementado manualmente.
  
### B) Selección de variables

Al igual que en el caso anterior, se busca que el alumno intente **mejorar** los resultados obenidos utilizando clasificadores base o ensembles al aplicar técnicas de selección de variables.

* **B.1 Utilizar un método filter basado en rankings para evaluar distintos subconjuntos**
* **B.2 Implementar al menos un algoritmo de búsqueda wrapper**
* **B.3 (Opcional) Implementar CFS**


### Consejos

En esta tarea se valorará muy positivamente que se realice de manera correcta el aprendizaje y la evaluación identificando distintos conjuntos de entrenamiento y test así como usando validación cruzada. Los resultados se contrastarán para los datos pima, wisconsin y titanic, siendo obligatorio **realizar al menos una subida al servidor de los resultados obtenidos para titanic**.

Se valorará muy positivamente también el uso de tablas y gráficas para exponer los resultados.

En el caso de la selección de variables se valorará también exponer claramente comparativas entre las distintas métricas obtenidas por los atributos para corroborar si existen diferencias entre métodos wrapper y filter.

---------------------------------------

## Uso y ajuste de métodos basados en ensembles

En este caso lo que se realizara es adaptar el selector de las variables que se ha implementados en la práctica 2 a los nuevos modelos que vamos a ver para después realizar el análisis de los atributos y su selección comparando los resultados con los obtenidos en las primeras selecciones.

Lo primero que vamos a realizar, igual que en la práctica 2, es importar los modulos necesarios para realizar esta práctica y las configuraciones necesarias para los mismo.

In [1]:
# Antes de empezar es fundamental cargar las librerias de python!
import numpy as np
import pandas as pd
from ggplot import *
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

import seaborn as sns
sns.set(color_codes=True)
# Configuracion para seaborn
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

/usr/local/lib/python2.7/dist-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/usr/local/lib/python2.7/dist-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


También, se ha creado una función, que se llama envioPrediccion, utilizada para enviar los resultados al servidor de minikaggle que se nos ha proporcionado para esta práctica en el anexo (AnexoMiniKaggle). En esta, se realiza las importaciones, la carga y el tratamiento de los DataSet por defecto proporcionado en el mismo anexo y la funcion que realiza el envio de la predicion al servidor para su validación, la cual se le ha pasado por parametro a la misma.

In [2]:
import pycurl, io, json

semilla = 3955
df_val = pd.read_csv("./data/titanic_validacion.csv")
df_val = df_val.drop("Name", 1)
df_val = df_val.drop("PassengerId",1)
df_train = pd.read_csv("./data/titanic_train.csv")
df_train = df_train.drop("Name", 1)
df_train = df_train.drop("PassengerId",1)
atrib_train = df_train.drop('label', 1)
clase_train = df_train['label']


def envioPrediccion(prediccion):
    user       = 'Ruben.Donate@alu.uclm.es'
    passwd     = 'TNRQB7ZO'
    # Este código recibe directamente el vector de predicciones y lo convierte a una
    # String separando los valores por comas
    pred_str   = ','.join(prediccion)
 
    url  = "http://212.47.226.96/predict"
    data = json.dumps({'user':user, 'passwd':passwd, 'prediction':pred_str})
    
    out = io.BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(pycurl.HTTPHEADER, ['Content-Type: application/json'])
    c.setopt(pycurl.POST, 1)
    c.setopt(c.POSTFIELDS, data)
    c.setopt(c.WRITEFUNCTION, out.write)
    
    c.perform()
    
    response = json.loads(out.getvalue().decode())
    print("Response: ",response)
    out.close()

Por ultimo, se ha creado una función para la visualización de los resultados en formato HTML en forma de tabla para que sea mas comoda su visualización. Esta función se ha creado tomando como base la que se proporciono en la práctica 3 pero con la peculiaridad que se ha generalizado para poder mostar listas que contengan los elementos que sean necesarios de manera automática.

In [3]:
from IPython.display import display, HTML

def printTable(list):
    table = """<table>%s</table>"""
    row = """<tr>%s</tr>"""
    cell = """<td>%s</td>"""
    report =  table % ''.join([row % ''.join([(cell % xi) for xi in x]) for x in list])
    display(HTML(report))

A continuación, se empezará a ver las implementaciones que he realizado para el método de selección que he diseñado. Siendo el primero punto que se explicara el proceso de pretratamiento que se le realiza a los a clasificar.

### Preprocesador de los datos

Este paso es el mas importante en un proceso de Minería de Datos. Esto es debido a que según el pretratamiento de los datos se pueden obtener resultados distintos. El problema es que se tiene realizar el pretratamiento de los datos de manera generica para que pueda utilizarse con cualquier DataSet que se nos proporcione. Esto hace que solo se pueda tratar los valores pedidos que pueda tener el DataSet, ya que como se va a trabajar con árboles no es necesario la normalización de los valores de los mismos, conforme se ha podido comprobar en la práctica 2. Esto hace que la función limpiaDataSet unicamente rellene los valores perdidos que pueda tener el DataSet con el valor de la mediana obtenido para ese atributo y ese DataSet.

In [4]:
def limpiaDataSet (df):
    df_limpio = df.fillna(df.median())
    return df_limpio

También, se ha creado una función que se encarga de devolver los atributos y la clase, la cual se identifica mediante una la etiqueta que es pasada por paramentro, separados de un DataSet que también es pasado por parametro.

In [5]:
def separarAtributosClase(df,etiqueta):
    dfLimpio = limpiaDataSet(df)
    atributos = dfLimpio.drop(etiqueta, 1)
    clase = dfLimpio[etiqueta]
    return atributos, clase

En este punto, los siguiente que nos nos toca es carga los datos con los que vamos a trabajar.

### Carga de los Dataset de Training

En este punto, cargaremos los DataSet con los que será posible operar para su clasificación.

#### Base de Datos Iris

In [6]:
irisdf = pd.read_csv("./data/iris.csv")

#### Base de Datos Pima

In [7]:
pimadf = pd.read_csv("./data/pima.csv")

#### Base de Datos Wisconsin

Para este DataSet, además de cargar el DataSet se ha procedido a eliminar un atributo que no aportaba información como es el Id del paciente, al ser distinto para cada paciente.

In [8]:
wisconsindf = pd.read_csv("./data/wisconsin.csv")
wisconsindf = wisconsindf.drop("patientId", 1)

#### Base de Datos de Titanic

En un principio, se establece que los DataSets para trabajar eran **iris, pima y wisconsin**. Pero posteriormente se nos proporciono el DataSet Titanic. Este último, es a partir del cual se va a evaluar la predicción y el modelo, mediante un conjunto de valores no proporcionados para entrerar el modelo. Por ese motivo, se ha escogido realizar todo el estudio sobre este DataSet, aunque la implementación al ser generica es completamente funcional con cualquiera de los otros DataSet proporcionado. 

In [9]:
titanicdf = df_train

### Cross Validation

Como se nos pedía que en los experimentos utilizaramos validación cruzada para obtener los resultados, por ese motivo he creado una función en la que se le pasa como parametro un modelo, un DataSet, la etiqueta que identifica a la clase, las partes en las que se va a dividir el DataSet, la metrica que se desea que nos devuelva, y una semilla. Todo esto para que la función poceda a devolvernos el array que ha obtenido de aplicar la validación cruzada con los parametros que se le han pasado.

In [10]:
def crossValidation(modelo, atributos, clase, partes, medida, semilla):
    np.random.seed(seed=semilla)
    scores = cross_val_score(modelo, atributos, clase, cv=partes, scoring=medida)
    return scores

En este punto ya hemos terminado, con los tareas comunes y comenzaré con la implementación de mi ensembler.

### Ensembler Propio

Un ensembler es un módelo que construye otros modelos diferentes para posteriormente agregar los resultados obtenido, para así intentar reducir la varianza e intentar mejor los resultados.

Para mi implementación del Ensembler, he creado una función que realiza un muestreo del DataSet con el que trabajamos y otra que es el propio ensembler, las cuales explicare a continuación.

#### Muestreo con Reemplazo

Esta función se ha creado para generar una muestra del DataSet que se el pasa por parametro con el tamaño que también se le pasa por parametro y una semilla. También se le pasa por parametro si en el muestro se le permite o no realizar reemplazo cuando se selecciona una ocurrencia del DataSet. He decidido realizarlo así para poder tenerlo controlado lo maximo posible para a continuación poder usarlo comodamente en mi ensembler.

In [11]:
def seleccionMuestra(df,tamano,reemplazo,semilla):
    np.random.seed(semilla)
    elegido = df.sample(n=tamano,random_state=semilla,replace=reemplazo)
    return elegido

#### Ensembler Propio

Para mi ensembler, he decido utilizar los árboles de decisión vistos en la práctica 2 como modelo para este. Por ello lo que se realizar es limpiar el DataSet, fijar el valor de la semilla para obtener número aleatorios, se inicializan resultadosPedicion donde se almacena la predicción las los elementos a clasificar, muestas donde se almacena la muestras utilizadas para construir cada uno de los modelo, predicciones donde se almacena la predicción que realiza cada uno de los modelos que se construyen y resultadoEnsembler donde se almacena la información que hemos generado durante todo el proceso, como puede ser la muestra utilizada, el arbol construido, la predicción realizada y los resultados que obtiene dicho arbol. A continuación, lo que se realiza es selecionar una semilla para le modelo, obtener una muestra del DataSet original. A partir de esta muestra se construyele el árbol de decisión, se entrena con los datos de la muestra generada y por ultimo se genera la predicción para los datos que deseamos clasificar. Esto es repetido tantas veces como modelos deseamos que nos genere. Para terminar, realizar una agregación de los resutlados obtenidos en cada uno de los módelos utilizando la técnica del voto por la mayoría.

In [12]:
def modeloEnsebler(df,tamanoMuestra,reemplazo,test,numModelos,criterio,profundidad,minEjemplos,poda,etiqueta,partes,medida,seed):
    df = limpiaDataSet(df)
    np.random.seed(seed)
    resultadoPrediccion=[0 for i in range(len(test))]
    muestras=[]
    predicciones=[[0 for j in range(numModelos)] for i in range(len(test))]
    metricas=[]
    resultadoEnsembler=[]
    for i in range (numModelos):
        seedModel=int(10000*np.random.random())
        muestra = seleccionMuestra(df,tamanoMuestra,reemplazo,seedModel)
        muestras.append(muestra)
        arbolParaEnsembler = modeloArbol(muestra,etiqueta,criterio,profundidad,minEjemplos,poda,partes,medida,seedModel)
        atributos, clase = separarAtributosClase(muestra,etiqueta)
        metricas.append(arbolParaEnsembler[2])
        clasificador = arbolParaEnsembler[0].fit(atributos,clase)
        prediccion = clasificador.predict(test)
        resultadoEnsembler.append((muestra,arbolParaEnsembler[0],prediccion,arbolParaEnsembler[1],arbolParaEnsembler[2]))
        for k in range(len(prediccion)):
            predicciones[k][i]=prediccion[k]
    for j in range(len(predicciones)):
        resultadoPrediccion[j]=mode(predicciones[j]).mode[0]
    return (resultadoEnsembler,resultadoPrediccion,metricas,np.mean(metricas))

Además, como en mi algoritmo de selección para automatizarlo en un bucle he creado una función de orden superior que me devuelva la función de este modelo.

In [13]:
def modeloEnseblerSup():
    return modeloEnsebler

### Modelos Genericos

A continuación creare el resto de los módelos genericos de ensembler que hemos estudiado. Todos estos módelos también han sido creado utilizando el orden superior para su posterior tratamiento en el proceso de seleccion de los módelos.

#### Bargging

La estrategia básica tras el algoritmo de **bagging** es la agregación de distintos clasificadores que han sido aprendidos mediante técnicas de muestreo con remplazo.

In [14]:
def modeloBaggingSup(modeloBase):
    def modeloBagging(df,tamanoMuestra,reemplazo,test,numModelos,criterio,profundidad,minEjemplos,poda,etiqueta,partes,medida,seed):
        df = limpiaDataSet(df)
        resultadoEnsembler = BaggingClassifier(modeloBase(criterio,profundidad,minEjemplos,poda,seed),n_estimators=numModelos,max_samples=(tamanoMuestra/len(df)),bootstrap=reemplazo,random_state=seed)
        atributos, clase = separarAtributosClase(df,etiqueta)
        metrica = crossValidation(resultadoEnsembler,atributos,clase,partes,medida,seed)
        clasificador = resultadoEnsembler.fit(atributos,clase)
        resultadoPrediccion = clasificador.predict(test)
        return (resultadoEnsembler,resultadoPrediccion,metrica,metrica.mean())
    return modeloBagging

#### Boosting

La estrategia de boosting es completamente diferente a la anterior, ya que en lugar de reducir la varianza al aprender múltiples clasificadores independientes, realiza un proceso iterativo en el que sobreajustaremos los datos en aquellas instancias más complicadas de aprender para nuestro modelo. 

In [15]:
def modeloBoostingSup(modeloBase):
    def modeloBoosting(df,tamanoMuestra,reemplazo,test,numModelos,criterio,profundidad,minEjemplos,poda,etiqueta,partes,medida,seed):
        df = limpiaDataSet(df)
        resultadoEnsembler = AdaBoostClassifier(modeloBase(criterio,profundidad,minEjemplos,poda,seed),n_estimators=numModelos,random_state=seed)
        atributos, clase = separarAtributosClase(df,etiqueta)
        metrica = crossValidation(resultadoEnsembler,atributos, clase,partes,medida,seed)
        clasificador = resultadoEnsembler.fit(atributos,clase)
        resultadoPrediccion = clasificador.predict(test)
        return (resultadoEnsembler,resultadoPrediccion,metrica,metrica.mean())
    return modeloBoosting

#### Random Forrest

Una alternativa muy popular al bagging es el clasificador random forest. En este caso también se utiliza muestreo con remplazo, pero adicionalmente integra otras técnicas de regularización en el aprendizaje de los árboles de decisión para ampliar la generalización del ensemble.

Un clasificador random forest siempre utiliza árboles de decisión como submodelos y por ello no requiere un clasificador base para su definición.

In [16]:
def modeloRandomForrestSup():
    def modeloRandomForrest(df,tamanoMuestra,reemplazo,test,numModelos,criterio,profundidad,minEjemplos,poda,etiqueta,partes,medida,seed):
        df = limpiaDataSet(df)
        resultadoEnsembler = RandomForestClassifier(n_estimators=numModelos,criterion=criterio,max_depth=profundidad,min_samples_split=minEjemplos,min_impurity_split = poda,bootstrap=reemplazo,n_jobs=numModelos,random_state=seed)
        atributos, clase = separarAtributosClase(df,etiqueta)
        metrica = crossValidation(resultadoEnsembler,atributos, clase,partes,medida,seed)
        clasificador = resultadoEnsembler.fit(atributos,clase)
        resultadoPrediccion = clasificador.predict(test)
        return (resultadoEnsembler,resultadoPrediccion,metrica,metrica.mean())
    return modeloRandomForrest

####  Gradient Boosting

Es una generalización del algoritmo de boosting que utiliza funciones de coste concretas en referencia al proceso de aprendizaje de árboles. Actualmente es una de las técnicas más exitosas para resolver problemas de clasificación complejos.

In [17]:
def modeloGradientBoostingSup():
    def modeloGradientBoosting(df,tamanoMuestra,reemplazo,test,numModelos,criterio,profundidad,minEjemplos,poda,etiqueta,partes,medida,seed):
        df = limpiaDataSet(df)
        resultadoEnsembler = GradientBoostingClassifier(n_estimators=numModelos,max_depth=profundidad,min_samples_split=minEjemplos,min_impurity_split = poda,random_state=seed)
        atributos, clase = separarAtributosClase(df,etiqueta)
        metrica = crossValidation(resultadoEnsembler,atributos, clase,partes,medida,seed)
        clasificador = resultadoEnsembler.fit(atributos,clase)
        resultadoPrediccion = clasificador.predict(test)
        return (resultadoEnsembler,resultadoPrediccion,metrica,metrica.mean())
    return modeloGradientBoosting

#### Arbol de Decision

Lo que se realiza es crear un método aprovechando el orden superior que lo utilizerán los ensembler para la construcción de sus modelos que utilizaran para realizar la clasificación.

In [18]:
def modeloArbolSup():
    def modeloArbolS(criterio,profundidad,minEjemplos,poda,seed):
        modelo= tree.DecisionTreeClassifier(criterion = criterio, max_depth = profundidad, min_samples_split = minEjemplos, min_impurity_split = poda, random_state = seed)
    return modeloArbolS

Además se utiliza el mismo método que se implento en la práctica 2 para construir y entrenar un árbol de decisión. Esto es necesario ya que el proceso selección toma como limite maximo de profundidad la que obtiene la que obtiene de un árbol con la configuración por defecto.

In [19]:
def modeloArbol (df,etiqueta,criterio,profundidad,minEjemplos,poda,partes,medida,seed):
    df = limpiaDataSet(df)
    modelo= tree.DecisionTreeClassifier(criterion = criterio, max_depth = profundidad, min_samples_split = minEjemplos, min_impurity_split = poda, random_state = seed)
    atributos, clase = separarAtributosClase(df,etiqueta)
    metrica = crossValidation(modelo,atributos, clase,partes,medida,seed)
    return (modelo,metrica,metrica.mean())

### Selector de Modelos

Este selector de modelos es una adaptación del modelo de selección de modelos para los árboles de decisión que se implemento en la práctica 2 pero adaptandolo a los ensembler y abstrayendolo en una función.

Este método lo que realizar es construir un árbol con una configuración establecida con los valores por defecto para después buscar la profundidad y criterio optimo para el árbol que buscamos. Para realizar esta busqueda lo que realiza es realizar una busqueda por bisección de la profundidad y probar cual de los dos tipos de criterios utilizados es mejor.

In [20]:
def optimizacionParametros(df,modeloEmseblerOpt,minEjemplos,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio,poda,etiqueta,partes,medida,seed):
    profundidad = None
    ensemblerArboles={}
    modeloMejorEnsemblerArbol=()
    atrib_train, clase_train=separarAtributosClase(df,etiqueta)
    arbol = modeloArbol(df,etiqueta,criterio[0],profundidad,minEjemplos,poda,partes,medida,seed)
    clasificador = arbol[0].fit(atrib_train , clase_train)
    min_profundidad = 1
    max_profundidad = clasificador.tree_.max_depth
    modeloMejorEnsemblerArbol = modeloEmseblerOpt(df,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio[0],max_profundidad,minEjemplos,poda,etiqueta,partes,medida,seed)
    ensemblerArboles[(criterio[0],max_profundidad,minEjemplos,poda)]=modeloMejorEnsemblerArbol
    profundidad =int((max_profundidad+min_profundidad)/2)
    ensemblerArbol=modeloEmseblerOpt(df,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio[0],profundidad,minEjemplos,poda,etiqueta,partes,medida,seed)
    ensemblerArboles[(criterio[0],profundidad,minEjemplos,poda)]=ensemblerArbol
    while ((modeloMejorEnsemblerArbol[3]<ensemblerArbol[3])and(max_profundidad-min_profundidad>1))and(profundidad>0):
        modeloMejorEnsemblerArbol=ensemblerArbol
        max_profundidad=profundidad
        profundidad =int((max_profundidad+min_profundidad)/2)
        for c in criterio:
            aux_ensemblerArbol1=modeloEmseblerOpt(df,tamanoMuestra,reemplazo,valoresTest,numModelos,c,profundidad,minEjemplos,poda,etiqueta,partes,medida,seed)
            ensemblerArboles[(c,profundidad,minEjemplos,poda)]=aux_ensemblerArbol1
            if aux_ensemblerArbol1[3]>=ensemblerArbol[3]:
                ensemblerArbol=aux_ensemblerArbol1
            if ((modeloMejorEnsemblerArbol[3]>=ensemblerArbol[3])and(max_profundidad-profundidad>=0)):
                min_profundidad = profundidad
    return [modeloMejorEnsemblerArbol, ensemblerArboles]

### Resultado

En este punto, lo que realiza es inicializar la variables necesarias para construir los ensembler basandome en árboles de decisión. Una de la variables es un lista de los módelos que va a tener que generar. Por ultimo, recorre esta lista para generar y obtener los resultados de cada uno de los modelos y mostrarlos de una tabla.

In [21]:
seed = 3568
etiqueta = "label"
medida = "accuracy"
partes = 10
minEjemplos=2
df = titanicdf
criterio = ("entropy","gini")
poda = 1e-7
numModelos=100
tamanoMuestra=len(df)
reemplazo=True
valoresTest=df_val
resultado={}
listaAcurracy=[]
modeloEmseblerOpt=(("Propio", modeloEnseblerSup()),("Bagging",modeloBaggingSup(modeloArbolSup())),("Boosting",modeloBoostingSup(modeloArbolSup())),("RandomForrest",modeloRandomForrestSup()),("GradientBoosting",modeloGradientBoostingSup()))
mejorResultado=None
for tipo in modeloEmseblerOpt:
    auxResultado=optimizacionParametros(df,tipo[1],minEjemplos,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio,poda,etiqueta,partes,medida,seed)
    #print("Ensembler ",tipo[0])
    #%timeit -n 100 optimizacionParametros(df,tipo[1],minEjemplos,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio,poda,etiqueta,partes,medida,seed)
    listaAcurracy.append((auxResultado[0][3],tipo[0]))
    resultado[tipo[0]]= auxResultado
    if (mejorResultado==None) or (auxResultado[0][3]>mejorResultado[1][0][3]):
        mejorResultado=(tipo[0],auxResultado)
listaAcurracy=sorted(listaAcurracy,reverse=True)
listaAcurracy.insert(0,("Acurracy","Ensembler"))
printTable(listaAcurracy)

Acurracy,Ensembler
0.880419841881,Propio
0.813247863248,RandomForrest
0.81237143271,GradientBoosting
0.794567579313,Boosting
0.617992177314,Bagging


Se puede observar que el mejor de los resultados es el del ensembler que he implementado, a pesar de ser algo extraño ya que se esperaba que el Random Forrest fuera el mejor ya que suele obtener muy buenos resultados. He pensado que podría ser un problema con la poda pero después de probar el módelo sin podar se obtiene un resultado peor incluso del modelo obtenido para ese modelo en proceso de optimización. Por lo cual, he de confiar en los resultados que he obtenido.

In [22]:
dfRandomForrest = limpiaDataSet(df)
resultadoEnsemblerRandomForrest = RandomForestClassifier(n_estimators=numModelos,criterion="entropy",random_state=seed)
atributosRandomForrest, claseRandomForrest = separarAtributosClase(dfRandomForrest,etiqueta)
metricaRandomForrest = crossValidation(resultadoEnsemblerRandomForrest,atributosRandomForrest, claseRandomForrest,partes,medida,seed)
print "Resultado del Random Forrest sin podar ",metricaRandomForrest.mean()

Resultado del Random Forrest sin podar  0.798855570042


In [23]:
print "El mejor modelo es el ",mejorResultado[0]," y su valor es ",mejorResultado[1][0][3]

El mejor modelo es el  Propio  y su valor es  0.880419841881


---------------------------------------

## Seleccion de variables

A partir de ahora, se explicará el proceso de selección de variables que he implementado para los modelos. El estudio se ha realizado en todos los casos con el DataSet de Titanic, el cual se le asigna a la variable dfSeleccion utilizada a partir de ahora en cada uno de los procesos de selección.

In [24]:
dfSeleccion=df_train

A continuación, se explicarán varios procesos de selección de variables para concluir realizando un análisis de los resultados obtenidos.

### Métricas filter y métodos de ranking univariados

Consiste en evaluar los atributos del DataSet a partir de medidas estadisticas para obtener la relevación que tienen los atributos en este DataSet.

En mi caso he implementado dos métodos distintos. El primero es el proporcionado como ejemplo en la práctica que consiste en obtener la información mútua que tiene cada uno de los atributos del DataSet.

In [25]:
from sklearn.feature_selection import mutual_info_classif

seed=1234

atributos, clase = separarAtributosClase(dfSeleccion,'label')
score = list(mutual_info_classif(atributos, clase, random_state=seed))
nombres = list(atributos.columns)
ranks = sorted( list(zip(score, nombres)), reverse=True )
ranks.insert(0,("Ranking","Atributo"))
printTable(ranks)

Ranking,Atributo
0.153785168642,Sex
0.128746421116,Fare
0.0454132785017,Pclass
0.0376224747332,EmbarkedS
0.0350278283985,Parch
0.0332930029916,Age
0.0320793449863,EmbarkedQ
0.019016575141,SibSp
0.00906885540481,EmbarkedC


Se puede observar que los atributos que más información aportan respecto de la clase son el "Sex" y "Fare" con bastante diferencia con el resto.

El segundo método implementado lo que realiza es seleccionar K elementos mas relevantes según el módelo estadistico de independencia Chi². En mi caso no he indicado un valor para K con la opción all. Esto se ha realizado para que me sacará los datos para todos los atributos.

In [26]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

atributos, clase = separarAtributosClase(dfSeleccion,'label')
atributosNuevos = SelectKBest(chi2, k='all')
atributosNuevosLista = list(atributosNuevos.fit(atributos, clase).scores_)
nombres = list(atributos.columns)
ranks = sorted( list(zip(atributosNuevosLista, nombres)), reverse=True )
ranks.insert(0,("Ranking","Atributo"))
printTable(ranks)

Ranking,Atributo
6270.01586244,Fare
219.596945869,Sex
37.2381562882,EmbarkedC
36.494371089,Pclass
19.0298430141,Parch
14.5577102277,Age
10.5738095834,EmbarkedS
2.37361435178,SibSp
0.220244970245,EmbarkedQ


Al igual que lo ocurrido en la implementación anterior se puede observar que hay dos atributos que resaltan sobre todos los demas, siendo estos "Fare" y "Sex". Además se observar que en estas dos implementaciones estos dos atributos son los más relevante a la hora de clasificar este DataSet.

### Métodos de selección wrapper

Este método consiste en analizar la relevancia de los atributos para un módelo concreto. En esta técnica existe controversia si se ha de analizar sobre el mismo modelo que se va a aprender, ajustandose de manera especifica corriendo el riesgo de sobreajustar, o relizarlo sobre otro modelo completamente distinto, haciendolo de manera mas general pero corriendo el riesgo de obtener peores resultados.

En mi caso, me he decantado por realizar el análisis sobre el clasificar Random Forrest. Esta elección ha sido debida a que este módelo es generico y obtiene como norma general muy buenos resultados, además se puede comparar con la siguiente al estar basadas el mismo algoritmo.

Para mi implementación lo que realizo es crear un clasificador Random Forrest, el cual es usado en el modelo RFECV (Eliminación recursiva de atributos con validación cruzada). Para este modelo, la validación cruzada se utiliza realizando dos particiones de manera estratificada. Se ha escogido un valor para la validación cruzada tan pequeño par evitar que el conjunto de selección se puedan dar todos los casos posibles y por motivo no eliminará ningún atributo.

In [27]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

seed=1234

atributos, clase = separarAtributosClase(dfSeleccion,'label')

randomForest = RandomForestClassifier(n_estimators=50,random_state=seed, criterion="entropy")

rfecv = RFECV(estimator=randomForest, step=1, cv=StratifiedKFold(2, random_state=seed), scoring='accuracy')
rfecv.fit(atributos, clase)


nombres = list(atributos.columns)
ranks = sorted( list(zip(rfecv.ranking_, rfecv.grid_scores_, nombres)) )
ranks.insert(0,("Grupo","Ranking","Atributo"))
printTable(ranks)

Grupo,Ranking,Atributo
1,0.665534804754,Sex
1,0.710526315789,Pclass
1,0.748726655348,Age
1,0.782682512733,Fare
2,0.783531409168,SibSp
3,0.782682512733,Parch
4,0.776740237691,EmbarkedC
5,0.779286926995,EmbarkedS
6,0.779286926995,EmbarkedQ


Se observa que en el primer grupo  nos encontramos con dos atributos que hemos visto anteriormente en los métodos filter que eran muy relevates como son "Sex" y "Fare". Esto me hace pensar que estos dos atributos son los más relevantes y se apoya en "Pclass" y "Age" para mejor sus resultados. También se puede observar que "Fare" a pesar de ser el atributo que mas información aporta esta relacionado con los otros atributo, ya que al unirlo con los otros la mejora no es muy significativa.

### Métrica de Importancia en Random Forest

Por ultimo, se va analizar la importancia que tiene según el Random Forrest cada uno de los atributos para su su proceso de clasificacción y se va a comparar con los resultados obtenidos en el apartado anterior.

In [28]:
seed=1234

atributos, clase = separarAtributosClase(dfSeleccion,'label')

# Entrenamos el modelo
classifier = randomForest.fit(atributos, clase)

# Obtenemos el ranking de características
classifier.feature_importances_
nombres = list(atributos.columns)
ranks = sorted( list(zip(classifier.feature_importances_, nombres)), reverse=True )
ranks.insert(0,("Ranking","Atributo"))
printTable(ranks)

Ranking,Atributo
0.29650730303,Fare
0.275304992376,Age
0.223092456303,Sex
0.081539360865,Pclass
0.0497897077064,SibSp
0.0393302767205,Parch
0.0183533052628,EmbarkedC
0.0108073421178,EmbarkedS
0.00527525561815,EmbarkedQ


Se puede observar que los cuatro primeros atributos coinciden con los seleccionados en el apartado anterior. Además, se tiene que el atributo que mas información aporta es "Fare" al igual que ocurría en los métodos Filter. En cambio el según lo ha cambiado por "Age" segido muy de cerca por "Sex", esto es debido a que manera independiente "Sex" es mas relevante pero al analizarlo de manera conjunto con "Fare" obtiene mas importancia "Age".

### Análisis de Resultados

De los resultados obtenidos, de los procesos de selección de variables se puede deducir que los atributos más importes del DataSet son "Fare" y "Sex". Además, se puede observar que también es muy interesante "Age" y en menor medida "Pclass" al unirlas con los dos anteriores.

In [29]:
seed = 3568
etiqueta = "label"
medida = "accuracy"
partes = 10
minEjemplos=2
df = titanicdf
df = limpiaDataSet(df)
atributosSeleccionados=("Fare", "Sex","Age","Pclass")
for i in  list(atributos.columns):
    if not i in atributosSeleccionados:
        df=df.drop(i, 1)
criterio = ("entropy","gini")
poda = 1e-7
numModelos=100
tamanoMuestra=len(df)
reemplazo=True
valoresTest=df_val
for i in  list(atributos.columns):
    if not i in atributosSeleccionados:
        valoresTest=valoresTest.drop(i, 1)
resultado={}
listaAcurracyDict={}
for valor in listaAcurracy:
    listaAcurracyDict[valor[1]]=valor[0]
listaAcurracy2=[]
modeloEmseblerOpt=(("Propio", modeloEnseblerSup()),("Bagging",modeloBaggingSup(modeloArbolSup())),("Boosting",modeloBoostingSup(modeloArbolSup())),("RandomForrest",modeloRandomForrestSup()),("GradientBoosting",modeloGradientBoostingSup()))
mejorResultado=None
for tipo in modeloEmseblerOpt:
    auxResultado=optimizacionParametros(df,tipo[1],minEjemplos,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio,poda,etiqueta,partes,medida,seed)
    #print("Ensembler ",tipo[0])
    #%timeit -n 100 optimizacionParametros(df,tipo[1],minEjemplos,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio,poda,etiqueta,partes,medida,seed)
    listaAcurracy2.append((auxResultado[0][3],listaAcurracyDict[tipo[0]],tipo[0]))
    resultado[tipo[0]]= auxResultado
    if (mejorResultado==None) or (auxResultado[0][3]>mejorResultado[1][0][3]):
        mejorResultado=(tipo[0],auxResultado)
listaAcurracy2=sorted(listaAcurracy2,reverse=True)
listaAcurracy2.insert(0,("Acurracy Nuevo", "Acurracy Antiguo", "Ensembler"))
printTable(listaAcurracy2)

Acurracy Nuevo,Acurracy Antiguo,Ensembler
0.881458936693,0.880419841881,Propio
0.818361581921,0.813247863248,RandomForrest
0.816644936984,0.81237143271,GradientBoosting
0.800499782703,0.794567579313,Boosting
0.617992177314,0.617992177314,Bagging


Se puede observar que al eliminar las variables que no son relevante los resultados obtenidos han mejorar a pesar de que la información que majabamos era menor. Esto demuestra que no toda la información es relevaten a la hora realizar un modelo.

In [30]:
seed = 3568
etiqueta = "label"
medida = "accuracy"
partes = 10
minEjemplos=2
df = titanicdf
df = limpiaDataSet(df)
atributosSeleccionados=("Fare", "Sex","Age")
for i in  list(atributos.columns):
    if not i in atributosSeleccionados:
        df=df.drop(i, 1)
criterio = ("entropy","gini")
poda = 1e-7
numModelos=100
tamanoMuestra=len(df)
reemplazo=True
valoresTest=df_val
for i in  list(atributos.columns):
    if not i in atributosSeleccionados:
        valoresTest=valoresTest.drop(i, 1)
resultado={}
listaAcurracyDict={}
for valor in listaAcurracy:
    listaAcurracyDict[valor[1]]=valor[0]
listaAcurracy3=[]
modeloEmseblerOpt=(("Propio", modeloEnseblerSup()),("Bagging",modeloBaggingSup(modeloArbolSup())),("Boosting",modeloBoostingSup(modeloArbolSup())),("RandomForrest",modeloRandomForrestSup()),("GradientBoosting",modeloGradientBoostingSup()))
mejorResultado=None
for tipo in modeloEmseblerOpt:
    auxResultado=optimizacionParametros(df,tipo[1],minEjemplos,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio,poda,etiqueta,partes,medida,seed)
    #print("Ensembler ",tipo[0])
    #%timeit -n 100 optimizacionParametros(df,tipo[1],minEjemplos,tamanoMuestra,reemplazo,valoresTest,numModelos,criterio,poda,etiqueta,partes,medida,seed)
    listaAcurracy3.append((auxResultado[0][3],listaAcurracyDict[tipo[0]],tipo[0]))
    resultado[tipo[0]]= auxResultado
    if (mejorResultado==None) or (auxResultado[0][3]>mejorResultado[1][0][3]):
        mejorResultado=(tipo[0],auxResultado)
listaAcurracy3=sorted(listaAcurracy3,reverse=True)
listaAcurracy3.insert(0,("Acurracy Nuevo", "Acurracy Antiguo", "Ensembler"))
printTable(listaAcurracy3)

Acurracy Nuevo,Acurracy Antiguo,Ensembler
0.875404527114,0.880419841881,Propio
0.798000869187,0.81237143271,GradientBoosting
0.790388236998,0.813247863248,RandomForrest
0.777625669999,0.794567579313,Boosting
0.617992177314,0.617992177314,Bagging


Además se puede observar que a "Pclass" es la menos relevante de estas cuatro haciendo que si se elimina la variación sea minima.